In [2]:
import random
import numpy as np

In [1]:
# Parameters
POP_SIZE = 20
GENS = 100
MUT_RATE = 0.01
CROSS_RATE = 0.7

In [3]:
# Binary encoding length
SIGN_BIT = 1
INT_BITS = 6
FRAC_BITS = 6
TOTAL_BITS = SIGN_BIT + INT_BITS + FRAC_BITS

In [4]:
def binary_to_float(binary):
    sign = -1 if binary[0] == '1' else 1
    integer = int(binary[1:1+INT_BITS], 2)
    fraction = int(binary[1+INT_BITS:], 2) / (2**FRAC_BITS)
    return sign * (integer + fraction)

In [6]:
def float_to_binary(value):
    sign = '1' if value < 0 else '0'
    integer = bin(int(abs(value)))[2:].zfill(INT_BITS)
    fraction = bin(int((abs(value) - int(abs(value))) * (2**FRAC_BITS)))[2:].zfill(FRAC_BITS)
    return sign + integer + fraction

In [347]:
def crossover(parent1, parent2):
    child1 = []
    child2 = []
    for i in range(bit_len):
        if random.random() < 0.5:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])
    return ''.join(child1), ''.join(child2)

In [7]:
def fitness(value):
    return abs(2 * value**2 + 5 * value - 3)

def initial_population(size):
    return [''.join(random.choice('01') for _ in range(TOTAL_BITS)) for _ in range(size)]

def select(population):
    selected = sorted(population, key=lambda x: fitness(binary_to_float(x)))
    return selected[:POP_SIZE//2]

def crossover(parent1, parent2):
    if random.random() < CROSS_RATE:
        point1 = random.randint(1, TOTAL_BITS - 1)
        point2 = random.randint(1, TOTAL_BITS - 1)
        if point1 > point2:
            point1, point2 = point2, point1
        child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
        child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
        return child1, child2
    else:
        return parent1, parent2

def mutate(individual):
    individual = list(individual)
    for i in range(TOTAL_BITS):
        if random.random() < MUT_RATE:
            individual[i] = '1' if individual[i] == '0' else '0'
    return ''.join(individual)


In [349]:
def roulette_wheel_selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    pointer = random.random()
    cumulative_prob = 0
    for i, prob in enumerate(probabilities):
        cumulative_prob += prob
        if pointer <= cumulative_prob:
            return population[i]

In [8]:
# Genetic Algorithm
population = initial_population(POP_SIZE)
for generation in range(GENS):
    selected = select(population)
    next_population = []
    while len(next_population) < POP_SIZE:
        parent1, parent2 = random.choice(selected), random.choice(selected)
        child1, child2 = crossover(parent1, parent2)
        next_population.append(mutate(child1))
        next_population.append(mutate(child2))
    population = next_population


In [9]:
# Find the best solution
best_individual = min(population, key=lambda x: fitness(binary_to_float(x)))
best_value = binary_to_float(best_individual)
best_fitness = fitness(best_value)


In [10]:
print(f'Best value: {best_value}, Fitness: {best_fitness}')

Best value: -2.984375, Fitness: 0.10888671875
